In [1]:
from utilities import database as db
import pandas as pd
import numpy as np
import requests
import yfinance as yf
from datetime import datetime
import time
import logging

from config import FMP_SP500_CONSTITUENT_URL
from tqdm import tqdm


In [2]:
### Additional setings ###

# Logging
logging.basicConfig(
    filename="data_collection.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Get Historical Constituesnts of SP500
**NOTE:** returns periods when ticker (stock) was added into SP500 and revomed<br>
date_removed equals to *2262-01-01* in such cases when stock is currently in SP500.

In [3]:
def get_sp500_hist():
    resp = requests.get(FMP_SP500_CONSTITUENT_URL)

    if resp.status_code != 200:
        return None

    df = pd.DataFrame.from_records(resp.json())

    removes = df[df['removedTicker'] != ''][['date', 'removedTicker']].dropna(ignore_index=True)
    removes.columns = ['date_removed', 'Ticker']

    adds = df[df['symbol'] != ''][['date', 'symbol']].dropna(ignore_index=True)
    adds.columns = ['date_added', 'Ticker']

    rotations = adds.merge(removes, how='left', on=['Ticker'])
    rotations['date_removed'] = rotations['date_removed'].fillna('2262-01-01')
    rotations = rotations[rotations['date_added'] < rotations['date_removed']]
    rotations = rotations.sort_values(by=['Ticker','date_removed'], ascending=True)
    rotations = rotations.drop_duplicates(subset=['Ticker','date_removed'], keep='first', ignore_index=True)
    rotations['date_added'] = pd.to_datetime(rotations['date_added'], format='%Y-%m-%d')
    rotations['date_removed'] = pd.to_datetime(rotations['date_removed'], format='%Y-%m-%d')
    rotations.columns = [col.lower() for col in rotations.columns]
    return rotations


historical_constituents = get_sp500_hist()
historical_constituents

,date_added,ticker,date_removed
0,1988-10-06,AAL,1997-01-14
1,2015-03-20,AAL,2024-09-23
2,2015-07-08,AAP,2023-08-24
3,1982-11-18,AAPL,2262-01-01
4,1957-03-03,ABA,1973-05-31
...,...,...,...
1385,2001-08-06,ZBH,2262-01-01
1386,2019-12-20,ZBRA,2262-01-01
1387,2001-06-22,ZION,2024-03-15
1388,1982-03-11,ZRN,1995-12-19


### Write into the DB

In [4]:
historical_constituents.to_sql('sp500_hist_constituents', db.get_engine(), if_exists='replace', index=False)

390

### Read from the DB

In [28]:
df = pd.read_sql('SELECT * from sp500_hist_constituents', db.get_engine())
df

,date_added,ticker,date_removed
0,1988-10-06,AAL,1997-01-14
1,2015-03-20,AAL,2024-09-23
2,2015-07-08,AAP,2023-08-24
3,1982-11-18,AAPL,2262-01-01
4,1957-03-03,ABA,1973-05-31
...,...,...,...
1385,2001-08-06,ZBH,2262-01-01
1386,2019-12-20,ZBRA,2262-01-01
1387,2001-06-22,ZION,2024-03-15
1388,1982-03-11,ZRN,1995-12-19


# Collect Stock Prices
**NOTE:** Script takes all stocks seen in SP500 at least ones and requests data from YahooFinance (old stocks may not exist in YF database)

In [ ]:
def fetch_stock_prices(ticker, start_date=None, end_date=None, interval="1d"):
    stock = yf.Ticker(ticker)

    try:
        if end_date is None:
            data = stock.history(period="max", interval=interval)
        else:
            data = stock.history(start=start_date, end=end_date, interval=interval)
    except:
        return None
    data.reset_index(inplace=True)
    data['ticker'] = ticker
    data.drop_duplicates(subset=['Date'])
    data['Date'] = pd.to_datetime(data['Date']) 
    data.columns = [col.lower().replace(' ', '_') for col in data.columns]
    return data

def get_sp_500_prices():

    tickers_query = '''
    SELECT DISTINCT ticker as Ticker
    FROM sp500_hist_constituents as sph
    --WHERE ticker not in (
    --    SELECT DISTINCT ticker
    --    FROM sp500_constituents_hist_prices_raw)
    '''

    df = pd.read_sql(tickers_query, db.get_engine())
    tickers = list(df['ticker'].unique())

    no_data_tickers, data_tickers = [], []
    strategy = 'replace'

    for ticker in tqdm(tickers):
        data = fetch_stock_prices(ticker)

        if (data is not None) and (not data.empty):

            try:
                
                data.to_sql(
                    'sp500_constituents_hist_prices_raw',
                    db.get_engine(),
                    chunksize=1000,
                    if_exists=strategy,
                    index=False
                    )

                logging.info(f'[OK] - {ticker} - {data.shape[0]}')
                data_tickers.append(ticker)
            except:
                logging.info(f'[FAIL] - {ticker} - -1')
                no_data_tickers.append(ticker)
        else:
            logging.info(f'[FAIL] - {ticker} - 0')
            no_data_tickers.append(ticker)

        strategy = 'append'
        time.sleep(0.5)

    return {'OK' : data_tickers, 'FAILED' : no_data_tickers}



In [7]:
stats = get_sp_500_prices()

100%|██████████| 1339/1339 [1:39:07<00:00,  4.44s/it]   


In [10]:
print('Managed to get data:', len(stats['OK']))
print('FAILED to get data:', len(stats['FAILED']))

Managed to get data: 802
FAILED to get data: 537


# Collection of aditional Financial & MacroEconomic Indicators 
1) **VIX** - Volatility Indicator ([YFinance](https://finance.yahoo.com/quote/%5EVIX/))
2) **GOLD** - Gold price ([YFinance](https://finance.yahoo.com/quote/GC=F/))
3) **DXY** - US Dollar Index - measure of the value of the U.S. dollar relative to a basket of major foreign currencies - ([YFinance](https://finance.yahoo.com/quote/DX-Y.NYB/))
4) **Yield Curve Rates** - interest rates of bonds (usually government bonds) plotted across different maturities - ([source](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2024))
5) **FFER** - Federal Funds Effective Rate - the interest rate at which banks lend reserve balances to each other overnight in the US ([source](https://fred.stlouisfed.org/series/FEDFUNDS))

### SPY + VIX + GOLD + DXY

In [42]:
def get_add_indctrs_yf(list_of_tickers=[], start_date=None, end_date=None, table_name=None):
    strategy = 'replace'
    for ticker in tqdm(list_of_tickers):
        data = fetch_stock_prices(ticker, start_date, end_date)

        if (data is not None) and (not data.empty):
            try:
                data.to_sql(
                    table_name,
                    db.get_engine(),
                    chunksize=1000,
                    if_exists=strategy,
                    index=False
                    )

                logging.info(f'[OK] - {ticker} - {data.shape[0]}')
            except:
                logging.info(f'[FAIL] - {ticker} - -1')
        else:
            logging.info(f'[FAIL] - {ticker} - 0')
        time.sleep(0.5)
    

In [44]:
# Collects VIX, Gold and Dollar index data
get_add_indctrs_yf(list_of_tickers=['VIX', 'GC=F', 'DX-Y.NYB'], table_name='us_fin_indicators_raw')

# Collects SPY index data (SP500 index OHLCV)
get_add_indctrs_yf(list_of_tickers=['SPY'], table_name='spy_ohlcv_raw')

100%|██████████| 1/1 [00:05<00:00,  5.54s/it]


### Yield Curve Rates

In [43]:
def get_yield_curve(table_name='us_treasury_yield_curve_raw'):
    domain = 'https://home.treasury.gov'
    df_1 = pd.read_csv(f'{domain}/system/files/276/yield-curve-rates-1990-2024.csv')
    df_2 = pd.read_csv(f'{domain}/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2025/all?type=daily_treasury_yield_curve&field_tdr_date_value=2025&page&_format=csv')

    df_1['Date'] = pd.to_datetime(df_1['Date'], format='%m/%d/%y')
    df_2['Date'] = pd.to_datetime(df_2['Date'], format='%m/%d/%Y')

    data = pd.concat([df_2[list(df_1.columns)], df_1], axis=0, ignore_index=True)
    data = data.sort_values(by='Date', ascending=False, ignore_index=True)

    if (data is not None) and (not data.empty):
        try:
            data.columns = [col.lower().replace(' ', '_') for col in data.columns]
            data.to_sql(
                table_name,
                db.get_engine(),
                chunksize=1000,
                if_exists='replace',
                index=False
                )
        except:
            logging.info(f'[FAIL] - Could not collect YCR (Yield Curve Rates)')
    return data

In [41]:
ycr = get_yield_curve(table_name='us_treasury_yield_curve_raw')

print('Data shape:', ycr.shape)
ycr.head()

Data shape: (8846, 14)


,date,1_mo,2_mo,3_mo,4_mo,6_mo,1_yr,2_yr,3_yr,5_yr,7_yr,10_yr,20_yr,30_yr
0,2025-05-09,4.37,4.34,4.34,4.40,4.28,4.05,3.88,3.85,4.00,4.18,4.37,4.86,4.83
1,2025-05-08,4.37,4.35,4.34,4.40,4.28,4.05,3.90,3.85,4.00,4.18,4.37,4.86,4.83
2,2025-05-07,4.37,4.33,4.34,4.39,4.27,4.00,3.78,3.72,3.87,4.06,4.26,4.78,4.77
3,2025-05-06,4.37,4.33,4.33,4.38,4.25,3.98,3.78,3.73,3.90,4.10,4.30,4.82,4.81
4,2025-05-05,4.38,4.34,4.33,4.40,4.27,4.02,3.83,3.78,3.95,4.14,4.36,4.84,4.83


### Federal Funds Effective Rate

In [34]:
def get_ffer(table_name='us_fed_funds_rate_raw'):
    url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=FEDFUNDS&scale=left&cosd=1954-07-01'
    data = pd.read_csv(url)
    data.columns = ['date', 'fed_funds_rate']
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
    data = data.sort_values(by='date', ascending=False, ignore_index=True)

    if (data is not None) and (not data.empty):
        try:
            data.columns = [col.lower() for col in data.columns]
            data.to_sql(
                table_name,
                db.get_engine(),
                chunksize=1000,
                if_exists='replace',
                index=False
                )
        except:
            logging.info(f'[FAIL] - Could not collect YCR (Yield Curve Rates)')
    return data


In [36]:
ffer = get_ffer(table_name='us_fed_funds_rate_raw')

print('Data shape:', ffer.shape)
ffer.head()

Data shape: (850, 2)


,date,fed_funds_rate
0,2025-04-01,4.33
1,2025-03-01,4.33
2,2025-02-01,4.33
3,2025-01-01,4.33
4,2024-12-01,4.48
